In [0]:
%scala
spark.sparkContext.hadoopConfiguration.set(
  "fs.azure.account.key.bovianalytics.blob.core.windows.net",
  "a3/QhTcHik/gwnSCI/eBdjK1GruaFdeELpSzdnwKAKe6LebTUz6Ca4cKwUUWFdWO2JmUAFdaq/N6VMc/OXLdPg=="
)

In [0]:
from pyspark.sql.functions import to_date,col
from pyspark.sql.types import LongType, StringType, StructField, StructType, BooleanType, ArrayType, IntegerType, DoubleType, FloatType, TimestampType
import pyspark.sql.functions as F
from pyspark.sql import Window
from pprint import pprint as pp
import json
from urllib.request import  urlopen
import requests

#Set prior

##mean

In [0]:
PriorMeanDf = spark.createDataFrame(
    [
        (1,0.001835,-0.5000,29.96,34.11),
        (2,0.002745,-0.78,22.52,44.26),
        (3,0.002997,0.0000,22.54,48.41)
    ],
    ['parity','decay','offset','ramp','scale']  # add your column names here
)

PriorMeanDf = PriorMeanDf.\
    withColumn('means', F.struct(col('decay'),col('offset'),col('ramp'),col('scale'))).\
    select('parity','means')

PriorMeanDf.display()

parity,means
1,"List(0.001835, -0.5, 29.96, 34.11)"
2,"List(0.002745, -0.78, 22.52, 44.26)"
3,"List(0.002997, 0.0, 22.54, 48.41)"


##sd

In [0]:
PriorSdDf12 = spark.createDataFrame(
    [
        (1, 'sd',0.000738,0.002,1.530,6.891,1.91,1,'parity'),
        (2, 'sd',0.000979,0.007,0.9999,9.570,2.19,2,'parity')
        
    ],
    ['parity','type','decay','offset','ramp','scale','seMilk', 'equal','stratum']  # add your column names here
)

PriorSdDf12 = PriorSdDf12.\
    withColumn('sd', F.struct(col('decay'),col('offset'),col('ramp'),col('scale'))).\
    withColumn('discriminator',F.struct('equal','stratum')).\
    select('parity','discriminator','sd','seMilk')

PriorSdDf12.display()

parity,discriminator,sd,seMilk
1,"List(1, parity)","List(7.38E-4, 0.002, 1.53, 6.891)",1.91
2,"List(2, parity)","List(9.79E-4, 0.007, 0.9999, 9.57)",2.19


In [0]:
PriorSdDf3 = spark.createDataFrame(
    [
        (3, 'sd',0.000972,0.03,8.724,10.66,2.14,3,'parity')
        
    ],
    ['parity','type','decay','offset','ramp','scale','seMilk','lowerLimit','stratum']  # add your column names here
)

PriorSdDf3 = PriorSdDf3.\
    withColumn('sd', F.struct(col('decay'),col('offset'),col('ramp'),col('scale'))).\
    withColumn('discriminator',F.struct('lowerLimit','stratum')).\
    select('parity','discriminator','sd','seMilk')
               
PriorSdDf3.display()

parity,discriminator,sd,seMilk
3,"List(3, parity)","List(9.72E-4, 0.03, 8.724, 10.66)",2.14


##PriorTree

In [0]:
PriorTree12 = PriorMeanDf.\
    filter('parity!=3').\
    join(PriorSdDf12,'parity').\
    withColumn('priors',F.struct('means','sd','seMilk')).\
    withColumn('node',F.struct('priors')).\
    withColumn('content',F.struct('node')).\
    withColumn('priorsTree',F.struct('content','discriminator')).\
    select('priorsTree','parity')

PriorTree12.\
    display()

priorsTree,parity
"List(List(List(List(List(0.001835, -0.5, 29.96, 34.11), List(7.38E-4, 0.002, 1.53, 6.891), 1.91))), List(1, parity))",1
"List(List(List(List(List(0.002745, -0.78, 22.52, 44.26), List(9.79E-4, 0.007, 0.9999, 9.57), 2.19))), List(2, parity))",2


In [0]:
PriorTree3 = PriorMeanDf.\
    filter('parity=3').\
    join(PriorSdDf3,'parity').\
    withColumn('priors',F.struct('means','sd','seMilk')).\
    withColumn('node',F.struct('priors')).\
    withColumn('content',F.struct('node')).\
    withColumn('priorsTree',F.struct('content','discriminator')).\
    select('priorsTree','parity').\
    withColumn('ParityGroup',F.lit('3+'))

PriorTree3.display()

priorsTree,parity,ParityGroup
"List(List(List(List(List(0.002997, 0.0, 22.54, 48.41), List(9.72E-4, 0.03, 8.724, 10.66), 2.14))), List(3, parity))",3,3+


#Read data

In [0]:
# whole data
FittingData= spark. \
  read. \
  parquet('wasbs://gpluse-cluster-2@bovianalytics.blob.core.windows.net/Projects/ChenYoungYan/11022022/Output/FittingData/')

In [0]:
# 31693777
FittingData.count()

Out[8]: 31693777

In [0]:
FittingData = FittingData.\
    withColumn('HerdIdentifier',FittingData['HerdIdentifier'].cast(StringType())).\
    withColumn('AnimalIdentifier',FittingData['AnimalIdentifier'].cast(StringType()))


In [0]:
FittingData.printSchema()

root
-- HerdIdentifier: string (nullable = true)
-- AnimalIdentifier: string (nullable = true)
-- DaysInMilk: integer (nullable = true)
-- TestDayMilkYield: double (nullable = true)
-- LactationNumber: integer (nullable = true)
-- LastDIM: integer (nullable = true)

In [0]:
FittingData.show(2)

+--------------+----------------+----------+------------------+---------------+-------+
|HerdIdentifier|AnimalIdentifier|DaysInMilk|  TestDayMilkYield|LactationNumber|LastDIM|
+--------------+----------------+----------+------------------+---------------+-------+
|           503|             602|        93| 33.18257032542748|              5|    344|
|           503|             602|       207|19.090147572229576|              5|    344|
+--------------+----------------+----------+------------------+---------------+-------+
only showing top 2 rows



##Parity1&2

In [0]:
FittingData12 = FittingData.\
    filter('LactationNumber<3').\
    withColumnRenamed('HerdIdentifier','name').\
    withColumnRenamed('LactationNumber','parity').\
    withColumnRenamed('DaysInMilk','dim').\
    withColumnRenamed('TestDayMilkYield','milk').\
    withColumn('dayFraction', F.lit(1)).\
    withColumn('lacKey', F.concat(col('name'),F.lit("_"),col('AnimalIdentifier'), F.lit("_"), col('parity'))).\
    withColumn('Tuple', F.struct(col('dim'),col('milk'),col('dayFraction'))).\
    groupBy('name','AnimalIdentifier','lacKey','parity').\
    agg(F.array_sort(F.collect_list('Tuple')).alias('points')).\
    withColumn('lac', F.struct(col('lacKey'),col('parity'),col('points'))).\
    groupBy('name','AnimalIdentifier','parity').\
    agg(F.collect_list('lac').alias('lactations')).\
    withColumn('lactationSet', F.struct(col('name'),col('lactations'))).\
    withColumn('steppedFit',F.lit('true').cast(BooleanType())).\
    withColumn('returnInputData',F.lit('false').cast(BooleanType())).\
    withColumn('returnPriors',F.lit('false').cast(BooleanType())).\
    withColumn('returnPath',F.lit('false').cast(BooleanType())).\
    withColumn('fitMethod',F.lit('MilkBot@2.01')).\
    withColumn('preferredMilkUnit',F.lit('Kg')).\
    withColumn('options',F.struct(col('steppedFit'),col('returnInputData'),col('returnPriors'),col('returnPath'),col('fitMethod'),col('preferredMilkUnit'))).\
    join(PriorTree12,['parity']).\
    withColumn('fittingJob',F.struct(col('lactationSet'),col('options'),col('priorsTree'))).\
    withColumn('Json',F.struct(col('fittingJob'))).\
    select('name','AnimalIdentifier','parity','Json').\
    withColumn('json',F.to_json(col('Json')))

In [0]:
FittingData12.\
    show(2)

+----+----------------+------+--------------------+
|name|AnimalIdentifier|parity|                json|
+----+----------------+------+--------------------+
| 503|             670|     2|{"fittingJob":{"l...|
| 503|             678|     2|{"fittingJob":{"l...|
+----+----------------+------+--------------------+
only showing top 2 rows



In [0]:
# 62039
FittingData12.\
    count()

Out[16]: 62039

##Parity 3

In [0]:
FittingData3 = FittingData.\
    filter('LactationNumber>2').\
    withColumnRenamed('HerdIdentifier','name').\
    withColumnRenamed('LactationNumber','parity').\
    withColumnRenamed('DaysInMilk','dim').\
    withColumnRenamed('TestDayMilkYield','milk').\
    withColumn('dayFraction', F.lit(1)).\
    withColumn('lacKey', F.concat(col('name'),F.lit("_"),col('AnimalIdentifier'), F.lit("_"), col('parity'))).\
    withColumn('Tuple', F.struct(col('dim'),col('milk'),col('dayFraction'))).\
    groupBy('name','AnimalIdentifier','lacKey','parity').\
    agg(F.array_sort(F.collect_list('Tuple')).alias('points')).\
    withColumn('lac', F.struct(col('lacKey'),col('parity'),col('points'))).\
    groupBy('name','AnimalIdentifier','parity').\
    agg(F.collect_list('lac').alias('lactations')).\
    withColumn('lactationSet', F.struct(col('name'),col('lactations'))).\
    withColumn('steppedFit',F.lit('true').cast(BooleanType())).\
    withColumn('returnInputData',F.lit('false').cast(BooleanType())).\
    withColumn('returnPriors',F.lit('false').cast(BooleanType())).\
    withColumn('returnPath',F.lit('false').cast(BooleanType())).\
    withColumn('fitMethod',F.lit('MilkBot@2.01')).\
    withColumn('preferredMilkUnit',F.lit('Kg')).\
    withColumn('options',F.struct(col('steppedFit'),col('returnInputData'),col('returnPriors'),col('returnPath'),col('fitMethod'),col('preferredMilkUnit'))).\
    withColumn('ParityGroup',F.lit('3+')).\
    join(PriorTree3.drop('parity'),['ParityGroup']).\
    withColumn('fittingJob',F.struct(col('lactationSet'),col('options'),col('priorsTree'))).\
    withColumn('Json',F.struct(col('fittingJob'))).\
    select('name','AnimalIdentifier','parity','Json').\
    withColumn('json',F.to_json(col('Json')))


In [0]:
FittingData3.\
    show(2)

+----+----------------+------+--------------------+
|name|AnimalIdentifier|parity|                json|
+----+----------------+------+--------------------+
| 503|             602|     6|{"fittingJob":{"l...|
| 503|             604|     5|{"fittingJob":{"l...|
+----+----------------+------+--------------------+
only showing top 2 rows



In [0]:
# 50910
FittingData3.count()

Out[19]: 50910

## combined

In [0]:
FittingDataFinal = FittingData12.\
    unionAll(FittingData3).\
    cache()

In [0]:
FittingDataFinal.write.\
    mode("overwrite"). \
    parquet('wasbs://gpluse-cluster-2@bovianalytics.blob.core.windows.net/Projects/ChenYoungYan/11022022/Output/FinalDataMiel/')

In [0]:
FittingDataFinal = spark.read.\
  parquet('wasbs://gpluse-cluster-2@bovianalytics.blob.core.windows.net/Projects/ChenYoungYan/11022022/Output/FinalDataMiel/')

In [0]:
# should be 112949
FittingDataFinal.count()

Out[36]: 112949

# Fitting

Version Miel

In [0]:
headers = {
    'Content-type': 'application/json',
    'X-API-KEY': '00baNEuQtjm51nvL'
}

In [0]:
def getMilkbotData(row):
    response = requests.post('https://europe-west1-numeric-analogy-337601.cloudfunctions.net/milkBot-fitter/fitlactations', headers=headers, data=row)
    return response.text


In [0]:
""" Converting function to UDF """
milkbotUDF = udf(lambda z: getMilkbotData(z),StringType())


## herd by herd

In [0]:
FittingDataFinal = spark.read.\
  parquet('wasbs://gpluse-cluster-2@bovianalytics.blob.core.windows.net/Projects/ChenYoungYan/11022022/Output/0405Split/FinalDataMiel/')

In [0]:
FittingDataFinal.show(2)

+----+----------------+------+--------------------+
|name|AnimalIdentifier|parity|                json|
+----+----------------+------+--------------------+
|1121|            1202|     1|{"fittingJob":{"l...|
| 506|            1449|     3|{"fittingJob":{"l...|
+----+----------------+------+--------------------+
only showing top 2 rows



In [0]:
herd = FittingDataFinal.\
    filter('name!=1006').\
    select('name').distinct().\
    rdd.flatMap(lambda x: x).collect()

In [0]:
len(herd)

Out[39]: 90

In [0]:
herd

Out[37]: ['1090',
 '718',
 '740',
 '711',
 '987',
 '672',
 '979',
 '1112',
 '724',
 '1483',
 '974',
 '1021',
 '710',
 '512',
 '1032',
 '924',
 '966',
 '1109',
 '951',
 '787',
 '712',
 '1070',
 '973',
 '964',
 '726',
 '928',
 '1010',
 '656',
 '1071',
 '503',
 '1152',
 '1106',
 '1052',
 '505',
 '658',
 '1061',
 '736',
 '716',
 '1128',
 '678',
 '657',
 '1113',
 '645',
 '722',
 '1069',
 '983',
 '709',
 '1148',
 '1107',
 '927',
 '506',
 '1138',
 '1068',
 '725',
 '965',
 '1018',
 '929',
 '741',
 '1080',
 '731',
 '708',
 '1050',
 '654',
 '950',
 '643',
 '992',
 '642',
 '1048',
 '715',
 '513',
 '945',
 '1085',
 '1088',
 '514',
 '705',
 '1086',
 '1075',
 '720',
 '721',
 '735',
 '676',
 '1121',
 '677',
 '925',
 '1047',
 '504',
 '717',
 '707',
 '946',
 '920']

In [0]:
for a in herd:
    FittingDataFinal.\
    filter(col('name')==a).\
    withColumn("result", milkbotUDF(col("json"))).\
    repartition(10).\
    write.\
    mode('overwrite').\
    json('wasbs://gpluse-cluster-2@bovianalytics.blob.core.windows.net/Projects/ChenYoungYan/11022022/Output/0405Split/HerdByHerdFitting/' +a+'/')

# Read result

In [0]:
Result = spark.read.\
  json('wasbs://gpluse-cluster-2@bovianalytics.blob.core.windows.net/Projects/ChenYoungYan/11022022/Output/0405Split/HerdByHerdFitting/*')

In [0]:
Result.count()

Out[2]: 112949

In [0]:
Result.show(2)

+----------------+--------------------+----+------+--------------------+
|AnimalIdentifier|                json|name|parity|              result|
+----------------+--------------------+----+------+--------------------+
|            3033|{"fittingJob":{"l...| 504|     3|{"fits":[{"lacKey...|
|            2053|{"fittingJob":{"l...| 504|     2|{"fits":[{"lacKey...|
+----------------+--------------------+----+------+--------------------+
only showing top 2 rows



## format

In [0]:
from pyspark.sql import types as T

schema = T.ArrayType(T.StructType([
    T.StructField('lacKey', T.StringType()),
    T.StructField('n', T.IntegerType()),
    T.StructField('params', T.StructType([
        StructField('scale', FloatType(), True),
        StructField('ramp', FloatType(), True),
        StructField('decay', FloatType(), True),
        StructField('offset', FloatType(), True),
    ])),
    T.StructField('sdResiduals', T.FloatType()),
]))

ResultFormatted = Result.\
    select(F.json_tuple('result','fits').alias('FittingResult')).\
    withColumn('temp',F.explode(F.from_json('FittingResult',schema=schema))).\
    select('temp.*','*').\
    drop('temp','FittingResult').\
    withColumn('Magnitude',col('params.scale')).\
    withColumn('TimeToPeakYield',col('params.ramp')).\
    withColumn('Offset',col('params.offset')).\
    withColumn('Decay',col('params.decay')).\
    drop('params').\
    withColumn('HerdIdentifier', F.split('lacKey','_').getItem(0)).\
    withColumn('AnimalIdentifier', F.split('lacKey','_').getItem(1)).\
    withColumn('LactationNumber', F.split('lacKey','_').getItem(2)).\
    drop('lacKey')

In [0]:
#The fitting result should have the same rows number with the test-day record number: 31,693,777
ResultFormatted.count()

Out[9]: 31693041

In [0]:
ResultFormatted.show(2)

+---+-----------+---------+---------------+----------+----------+--------------+----------------+---------------+
|  n|sdResiduals|Magnitude|TimeToPeakYield|    Offset|     Decay|HerdIdentifier|AnimalIdentifier|LactationNumber|
+---+-----------+---------+---------------+----------+----------+--------------+----------------+---------------+
|  1|        0.0|23.898785|        22.7552| -0.775189|0.00218413|           504|            3033|              3|
|  2|   4.266536| 28.33078|      22.772387|-0.7740984|0.00218413|           504|            3033|              3|
+---+-----------+---------+---------------+----------+----------+--------------+----------------+---------------+
only showing top 2 rows



In [0]:
ResultFormatted .\
  agg(F.count('*').alias('NumbersOfRecord'),
        F.countDistinct("HerdIdentifier").alias('NumbersOfHerds'),
        F.countDistinct("HerdIdentifier","AnimalIdentifier").alias('NumberOfCows')).\
    display()

NumbersOfRecord,NumbersOfHerds,NumberOfCows
31693041,91,42894


In [0]:
ResultFormatted.write.\
    mode("overwrite"). \
    parquet('wasbs://gpluse-cluster-2@bovianalytics.blob.core.windows.net/Projects/ChenYoungYan/11022022/Output/0405Split/FittingResult/')